In [ ]:
import os
root = './data/dermamnist'
os.makedirs(root, exist_ok=True)

# 1. 라이브러리 설치
!pip install medmnist torch torchvision tqdm --quiet # tqdm 추가 설치

# 2. 라이브러리 import
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as T
from torchvision import models
from medmnist import DermaMNIST
from tqdm import tqdm # tqdm 임포트

# 3. GPU 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device} ({torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'})")

# 4. 데이터 전처리(transform)
transform = T.Compose([
    T.ToTensor(),
    T.Normalize(mean=[.5, .5, .5], std=[.5, .5, .5])
])

# train, validation, test 데이터셋 (224x224)
train_set = DermaMNIST(split='train', transform=transform, download=True, size=224)
val_set = DermaMNIST(split='val', transform=transform, download=True, size=224)
test_set = DermaMNIST(split='test', transform=transform, download=True, size=224)

# 5. DataLoader
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32)
test_loader = DataLoader(test_set, batch_size=32)

# 학습 손실 데이터를 저장할 딕셔너리 초기화 (여러 학습률 실험을 위해 여기에 초기화)
training_loss_data = {}

# 실험할 학습률 리스트
learning_rates_to_experiment = [1e-1, 1e-2, 1e-3, 1e-4]
num_epochs = 50 # 각 학습률에 대한 에포크 수
batch_size = train_loader.batch_size # 사용된 배치 크기

for learning_rate in learning_rates_to_experiment:
    print(f"\n--- Training with Learning Rate: {learning_rate} ---")
    # 현재 하이퍼파라미터 정보 출력
    print(f"Hyperparameters:")
    print(f"  Learning Rate: {learning_rate}")
    print(f"  Number of Epochs: {num_epochs}")
    print(f"  Batch Size: {batch_size}")
    print(f"  Optimizer: Adam") # 사용된 옵티마이저 정보
    print(f"  Loss Function: CrossEntropyLoss") # 사용된 손실 함수 정보
    print("-" * 30)


    # 6. MobileNetV2 모델 정의 및 classifier 출력 수정 (각 학습률마다 새로 정의)
    # 사전 학습된 가중치를 사용하므로, 각 실험마다 모델을 새로 로드합니다.
    model = models.mobilenet_v2(pretrained=True)
    in_features = model.classifier[1].in_features
    model.classifier[1] = nn.Linear(in_features, 7)  # DermaMNIST는 7개 클래스
    model = model.to(device)

    # 7. 손실함수 / optimizer (각 학습률마다 새로 정의)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)

    # 현재 학습률에 대한 손실 리스트 초기화
    training_loss_data[learning_rate] = []

    # 8. 학습
    for epoch in range(1, num_epochs + 1):
        model.train()
        running_loss = 0
        running_correct = 0
        total = 0

        # tqdm을 사용하여 학습 배치의 진행 상태 표시
        train_loop = tqdm(train_loader, leave=False, desc=f"Epoch {epoch}/{num_epochs} (LR: {learning_rate})")
        for imgs, labels in train_loop:
            imgs = imgs.to(device)
            labels = labels.to(device).view(-1)

            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * imgs.size(0)
            running_correct += (outputs.argmax(1) == labels).sum().item()
            total += labels.size(0)

            # 프로그레스 바에 현재 배치 손실 정보 추가 (선택 사항)
            # train_loop.set_postfix(loss=loss.item())


        train_loss = running_loss / total
        training_loss_data[learning_rate].append(train_loss) # 에포크별 학습 손실 저장
        train_acc = running_correct / total

        # 검증
        model.eval()
        val_loss = 0
        val_correct = 0
        val_total = 0
        # 검증 배치는 프로그레스 바 없이 진행
        with torch.no_grad():
            for imgs, labels in val_loader:
                imgs = imgs.to(device)
                labels = labels.to(device).view(-1)
                outputs = model(imgs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * imgs.size(0)
                val_correct += (outputs.argmax(1) == labels).sum().item()
                val_total += labels.size(0)

        val_loss /= val_total
        val_acc = val_correct / val_total

        # 에포크 결과 출력 (프로그레스 바 아래에 표시됨)
        print(f"Epoch {epoch:2d} | "
              f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")


    # 9. 평가 (각 학습률 학습 완료 후 평가)
    print(f"\n--- Evaluation with Learning Rate: {learning_rate} ---")
    model.eval()
    test_correct = 0
    test_total = 0
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs = imgs.to(device)
            labels = labels.to(device).view(-1)
            outputs = model(imgs)
            preds = outputs.argmax(1)
            test_correct += (preds == labels).sum().item()
            test_total += labels.size(0)

    print(f"Test Accuracy (LR={learning_rate}): {test_correct / test_total:.4f}")

# 모든 학습률 실험 완료 후 최종 메시지
print("\n--- All Learning Rate Experiments Completed ---")

Using device: cuda (Tesla T4)

--- Training with Learning Rate: 0.1 ---
Hyperparameters:
  Learning Rate: 0.1
  Number of Epochs: 50
  Batch Size: 32
  Optimizer: Adam
  Loss Function: CrossEntropyLoss
------------------------------


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch  1 | Train Loss: 1.5579, Train Acc: 0.6601 | Val Loss: 1.0370, Val Acc: 0.6690


Epoch  2 | Train Loss: 0.9888, Train Acc: 0.6680 | Val Loss: 1.2599, Val Acc: 0.6690


Epoch  3 | Train Loss: 0.9761, Train Acc: 0.6692 | Val Loss: 1.1351, Val Acc: 0.6690


...

Epoch 12 | Train Loss: 1.0241, Train Acc: 0.6652 | Val Loss: 1.0383, Val Acc: 0.6690


Epoch 13 | Train Loss: 1.0249, Train Acc: 0.6632 | Val Loss: 1.0415, Val Acc: 0.6690


Epoch 14 | Train Loss: 1.0417, Train Acc: 0.6659 | Val Loss: 1.2306, Val Acc: 0.6690


Epoch 15 | Train Loss: 1.0243, Train Acc: 0.6672 | Val Loss: 1.0191, Val Acc: 0.6690


Epoch 16 | Train Loss: 1.1070, Train Acc: 0.6682 | Val Loss: 1.1551, Val Acc: 0.6690


학습 과정에서 기록된 `training_loss_data`를 사용하여 에포크별 학습 손실 그래프를 그립니다. 각 라인은 다른 학습률에 해당됩니다.

In [ ]:
import matplotlib.pyplot as plt

# training_loss_data 딕셔너리가 학습 코드 실행 후 생성되었다고 가정합니다.
# 예시 데이터 구조: {learning_rate: [epoch1_loss, epoch2_loss, ...]}
# 만약 training_loss_data가 아직 없다면, 이전 학습 코드를 먼저 실행해야 합니다.
if 'training_loss_data' in locals() and training_loss_data:
    plt.figure(figsize=(12, 6))

    for lr, losses in training_loss_data.items():
        epochs = range(1, len(losses) + 1)
        plt.plot(epochs, losses, marker='o', linestyle='-', label=f'LR: {lr}')

    plt.xlabel('Epoch')
    plt.ylabel('Training Loss')
    plt.title('Training Loss over Epochs for Different Learning Rates')
    plt.legend()
    plt.grid(True)
    plt.show()
else:
    print("training_loss_data를 찾을 수 없습니다. 먼저 학습 코드를 실행하여 데이터를 생성해주세요.")